<hr>

# Wave Generation

In [ ]:
from data_generators.utils import random_waves as rw
import numpy as np
import pandas as pd
import plotly.express as px

# rw.seed()

# y = rw.half_waves_fixed_freq_varying_amp(num_points=10000, periods=7, amplitude=(3.5, 8.5), phase=5.7, smooth=10, noise=0.1)
# y = rw.full_waves_fixed_freq_varying_amp(num_points=10000, periods=7, amplitude=(3.5, 8.5), phase=5.7, noise=0.1)

y = rw.sinusoidal_wave(num_points=10000, periods=7, amplitude=(5.5, 7.2), wave_mode='full', varying_mode='fixed', phase=5.7, noise=0.1)
# y = rw.sinusoidal_wave(num_points=10000, periods=7, amplitude=(5.5, 7.2), wave_mode='full', varying_mode='fixed')

print(f"num_points: {len(y)}")
# display(y.head())

fig = px.line(y=y, title='Random Wave Example')
fig.update_layout(
    xaxis_title='Index',
    yaxis_title='Amplitude',
    template='plotly_dark'
)
fig.show()

In [ ]:
from data_generators.utils import random_waves as rw

print(rw.equals_parts(1000, 3))
print(rw.random_parts(1000, 5))

<hr>

# Test Smart Grid with InfluxDB Writer


In [ ]:
from datetime import datetime, timedelta
from generators.smart_grid.new_york_smart_grid import NewYorkSmartGridStreamGenerator
from writers.influxdb_writer import InfluxdbWriter
from writers.file_writer import FileWriter

START_TIME = datetime.now() - timedelta(minutes=5)
END_TIME = datetime.now() + timedelta(minutes=15)

# Define the data generator
generator = NewYorkSmartGridStreamGenerator(
    interval=1.0,  # 1 second interval
    nrows=1000,  # number of rows to generate in each batch
    num_stations=32,  # number of stations
    start_time=START_TIME,  # start time
    end_time=END_TIME,  # end time (15 minutes from now)
    loopback=True,  # do not loop back
)

# Define the InfluxDB writer
influx_writer = InfluxdbWriter(
    url='http://localhost:8086',
    token='sudo-banana-404-not-found',
    org='perspective',
    bucket='smart_grid',
    measurement='new_york_smart_grid',
    timestamp_col='timestamp',
    tag_cols=['station_name', 'latitude', 'longitude'],
    field_cols=['energy_consumption', 'current', 'voltage']
    )

# Define a CSV writer
csv_writer = FileWriter(
    file_path='test.csv',
    mode='w',
    )

loop_cycles = (END_TIME - START_TIME).seconds
for _ in range(loop_cycles):
    df = generator.get_data()
    influx_writer.write(df)
    csv_writer.write(df)

influx_writer.close()
csv_writer.close()